In [ ]:
using Knet, Plots, JLD2, NBInclude
@nbinclude("02.mnist.ipynb")  # loads MNIST, defines dtrn,dtst,Atype,train,softmax,zeroone
ENV["COLUMNS"]=80         # column width for array printing
#broken Plots.plotlyjs()               # for interactive plots
Plots.scalefontsizes(1.5)

## Define linear model

In [ ]:
# The predict function returns a score for each class as a linear function of input x
function linear(w,x)
    y = w[1]*mat(x) .+ w[2]
end;

In [ ]:
# Initialize weights as a list of [ weightMatrix, biasVector ]
winit1(;std=0.01)=map(Atype, [ std*randn(10,784), zeros(10,1) ]);
w = winit1()

## Accuracy and zero-one loss

In [ ]:
# Grab a minibatch
x,y = first(dtst)
summary.((x,y))

In [ ]:
# Initialize random model and calculate predictions for one minibatch
Knet.seed!(9)           # for replicability
w = winit1()         # random weight matrix and a zero bias vector
ypred = linear(w,x)  # predict
Array(ypred)         # predictions are given as a 10xN score matrix

In [ ]:
y'  # correct answers are given as an array of integers

In [ ]:
# accuracy gives percentage of correct answers
accuracy(ypred,y)        # 2-arg version: accuracy on this batch of 100 with initial w

In [ ]:
accuracy(w,dtst,linear)  # 3-arg version: accuracy on the whole test dataset

In [ ]:
# zeroone loss (error) defined as 1 - accuracy
zeroone(w,data,model) = 1 - accuracy(w,data,model)
zeroone(w,dtst,linear)

## Softmax loss function

In [ ]:
# Calculate softmax (cross entropy, negative log likelihood) loss of a model with weights w for one minibatch (x,y)
# Predict specifies the function for model output: ypred = predict(w,x;o...)
softmax(w,x,y,predict; o...)=nll(predict(w,x;o...),y)

In [ ]:
# per-instance average softloss for the first test minibatch, should be close to -log(1/10)=2.3
softmax(w,x,y,linear)

In [ ]:
# Manual loss calculation
using SparseArrays
ypred=linear(w,x)
yp1 = exp.(ypred)
yp2 = yp1 ./ sum(yp1,dims=1)
yp3 = -log.(yp2)
yc1 = Array(sparse(y,1:100,1f0))
sum(Array(yp3).*yc1) / 100

In [ ]:
using Statistics
softmax(w,data,predict) = mean(softmax(w,x,y,predict) for (x,y) in data)
softmax(w,dtst,linear)  # per-instance average softmax loss for the whole test set

## Calculating the gradient using Knet

In [ ]:
# Automatically defined gradient for softloss
softgrad = grad(softmax)  # softgrad returns gradient wrt 1st arg w

In [ ]:
Knet.seed!(9)
w1 = winit1(std=0.1)  # use a larger std to get a larger gradient for this example
size.(w1)

In [ ]:
g1 = softgrad(w1,x,y,linear)  # gradient has the same size and shape as the first parameter
size.(g1)

## Checking the gradient using numerical approximation

In [ ]:
Array(g1[2])'  
# Meaning of gradient:
# If I move the last entry of w[2] by epsilon, loss will go up by 0.345075 epsilon!

In [ ]:
Array(w1[2])'

In [ ]:
softmax(w1,x,y,linear)

In [ ]:
w1[2][10] = 0.1   # to numerically check the gradient let's move the last entry by +0.1.
Array(w1[2]')

In [ ]:
softmax(w1,x,y,linear)  
# We see that the loss moves by +0.03 as expected.
# You should check all/most entries in your gradients this way to make sure they are correct.

## Checking the gradient using manual implementation

In [ ]:
# Manually defined gradient for softloss
function softgrad_manual(w,x,y,predict)
    x = mat(x)
    p = predict(w,x)
    p = p .- maximum(p,dims=1) # for numerical stability
    expp = exp.(p)
    p = expp ./ sum(expp,dims=1)
    q = oftype(p, sparse(convert(Vector{Int},y),1:length(y),1,size(p,1),length(y)))
    dJdy = (p - q) / size(x,2)
    dJdw = dJdy * x'
    dJdb = sum(dJdy,dims=2)
    Any[dJdw,dJdb]
end;

In [ ]:
g2 = softgrad_manual(w1,x,y,linear)

In [ ]:
isapprox(g1[1],g2[1],rtol=0.1)

In [ ]:
isapprox(g1[2],g2[2],rtol=0.1)

## Training (SGD) loop

In [ ]:
# Train model(w) with SGD and return a list containing w for every epoch
function train(w,data,predict; epochs=100,lr=0.1,o...)
    weights = Any[deepcopy(w)]
    for epoch in 1:epochs
        for (x,y) in data
            g = softgrad(w,x,y,predict;o...)
            update!(w,g,lr=lr)  # w[i] = w[i] - lr * g[i]
        end
        push!(weights,deepcopy(w))
    end
    return weights
end;

## Training the linear model and underfitting

In [ ]:
if !isfile("lin.jld2")
    Knet.seed!(1)
    @time weights = train(winit1(),dtrn,linear,lr=0.1)           # 31.1s
    @time trnloss = [ softmax(w,dtrn,linear) for w in weights ]  # 22.2s
    @time tstloss = [ softmax(w,dtst,linear) for w in weights ]  # 3.7s
    @time trnerr  = [ zeroone(w,dtrn,linear) for w in weights ]  # 20.6s
    @time tsterr  = [ zeroone(w,dtst,linear) for w in weights ]  # 3.4s
    weights = map(x->Array.(x),weights)
    @save "lin.jld2" weights trnloss tstloss trnerr tsterr
else
    @eval (@load "lin.jld2")
end
minimum(tstloss),minimum(tsterr)  # 0.2667, 0.0744

In [ ]:
plot([trnloss tstloss],ylim=(.0,.4),labels=[:trnloss :tstloss],xlabel="Epochs",ylabel="Loss") 
# Demonstrates underfitting: training loss not close to 0
# Also slight overfitting: test loss higher than train

In [ ]:
plot([trnerr tsterr],ylim=(.0,.12),labels=[:trnerr :tsterr],xlabel="Epochs",ylabel="Error")  
# this is the error plot, we get to about 7.5% test error, i.e. 92.5% accuracy

## Visualizing the learned weights

In [ ]:
for t in 10 .^ range(0,stop=2,length=10) #logspace(0,2,20)
    i = ceil(Int,t)
    w = weights[i]
    w1 = reshape(Array(w[1])', (28,28,1,10))
    w2 = clamp.(w1.+0.5,0,1)
    IJulia.clear_output(true)
    display(hcat([mnistview(w2,i) for i=1:10]...))
    display("Epoch $i")
    sleep(1) # (0.96^i)
end